In [ ]:
import os
import re
import csv

# 파일 경로 지정
folder_path = r'C:/Users/wosle/OneDrive/Desktop/Bok_Projet_woslek/뉴스 스크래피/머니 투데이/money_news_texts'

# CSV 파일 경로 설정 (결과 CSV 저장 위치)
csv_file = os.path.join(folder_path, 'C:/Users/wosle/OneDrive/Desktop/Bok_Projet_woslek/뉴스 클랜징/머니투데이/머니투데이클렌징4.csv')

# 🔹 삭제할 특정 단어 & 문장 패턴 (여기에 추가하면 자동 삭제됨)
copyright_patterns = [
    r"돈이 보이는 리얼타임 뉴스 머니투데이 무단전재 및",
    r"구독",
    r"머니투데이",
    r"돈이 보이는",
    r"리얼타임 뉴스",
    r"구독중",
    r"저작권자",
    r"재배포",
    r"Copyright",
    r"All rights reserved.",
    r"전재",
    r"AI 학습",
    r"이용 금지.",
    r"언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다.",
    r"기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다.",
    # 이메일 주소 패턴
    r'\S+@\S+\.\S+',  # 이메일 형식
    # 기자와 이름 제거: 예시 "홍길동 기자", "김창현 기자" 등
    r'\b[가-힣]+ 기자\b',  # "홍길동 기자" 형식 제거
    # "기자"와 이름 결합된 경우: 예시 "홍길동 기자", "김창현 기자"
    r'\b[가-힣]+(?: [가-힣]+)? 기자\b',  # 공백이 있을 경우도 처리
    # 사람 이름에 붙은 '씨' 삭제
    r'\b[가-힣]+(?: [가-힣]+)? 씨\b',  # "홍길동 씨", "김창현 씨"
    # 이름 뒤에 '기자'가 붙어있는 경우도 제거: 예시 "홍길동기자"
    r'\b[가-힣0-9]+기자\b',  # "홍길동기자", "000기자" 형식도 제거
]

# 날짜와 본문을 추출하는 함수
def extract_date_and_content(text):
    # 날짜 패턴: 'YYYY-MM-DD' 형식
    date_pattern = r'\d{4}-\d{2}-\d{2}'
    date_match = re.search(date_pattern, text)
    date = date_match.group(0) if date_match else ''
    
    # 본문에서 날짜를 제외하고 추출
    content = text.split('\n', 1)[1] if '\n' in text else text  # 첫 번째 줄 제외

    # 🔹 먼저 텍스트에서 "관련", "☞" 이후 텍스트 제거
    stop_keywords = ['[관련', '☞']
    for keyword in stop_keywords:
        content = re.split(rf'{re.escape(keyword)}', content)[0]  # 키워드 앞까지만 추출

    # 🔹 불필요한 공백 정리
    content = re.sub(r'\n+', ' ', content).strip()

    # 🔹 괄호 안의 내용 제거
    content = re.sub(r'\[.*?\]', '', content)  # 대괄호 제거
    content = re.sub(r'\(.*?\)', '', content)  # 소괄호 제거

    # 🔹 날짜(YYYY-MM-DD 형식) 제거
    content = re.sub(r'\d{4}-\d{2}-\d{2}', '', content)

    # 🔹 특수문자 제거 (구두점은 남김)
    content = re.sub(r'[^\w\s.!?]', '', content)

    # 🔹 특정 문구(단어 & 문장) 삭제
    for pattern in copyright_patterns:
        content = re.sub(pattern, '', content)

    # 불필요한 공백 정리
    content = re.sub(r'\s+', ' ', content).strip()

    return date, content

# CSV 파일에 결과 저장
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['date', 'title', 'id', 'content'])  # 헤더 추가

    # 폴더 내 모든 텍스트 파일 처리
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):  # .txt 파일만 처리
            file_path = os.path.join(folder_path, filename)

            # 파일 내용 읽기
            with open(file_path, encoding='utf-8') as f:
                text = f.read()

            # 날짜와 본문 추출
            date, content = extract_date_and_content(text)

            # 파일 이름에서 ID 추출 (예: news_2012-06-07_0002853937.txt에서 '0002853937' 추출)
            file_id = filename.split('_')[-1].replace('.txt', '')

            # 파일에서 첫 번째 줄을 제목으로 간주
            title = text.split('\n')[0]

            # 추출된 데이터 CSV에 기록
            writer.writerow([date, title, file_id, content])

print(f"Extracted data has been saved to {csv_file}")


Extracted data has been saved to C:/Users/wosle/OneDrive/Desktop/Bok_Projet_woslek/뉴스 클랜징/머니투데이/머니투데이클렌징4.csv
